In [6]:
# 更新数据
import re
import pandas as pd

csv_file = "Wuhan-2019-nCoV.csv"
json_file = "Wuhan-2019-nCoV.json"

df = pd.read_csv(csv_file)
df["date"] = df["date"].map(lambda x: "-".join([i.zfill(2) for i in re.split("\\D+", x)]))
df = pd.concat([df, cn_area_df, cn_global_df, cn_day_df], sort=False)
df["country"].fillna("", inplace=True)
df["countryCode"].fillna("", inplace=True)
df["province"].fillna("", inplace=True)
df["provinceCode"].fillna("", inplace=True)
df["city"].fillna("", inplace=True)
df["cityCode"].fillna("", inplace=True)
df["confirmed"] = df["confirmed"].fillna(0).astype(int)
df["suspected"] = df["suspected"].fillna(0).astype(int)
df["cured"] = df["cured"].fillna(0).astype(int)
df["dead"] = df["dead"].fillna(0).astype(int)
df.drop_duplicates(subset=["date", "country", "province", "city"], keep="last", inplace=True)
df.sort_values(["date", "country", "province", "city"], na_position="first", inplace=True)
df.to_csv(csv_file, index=False, encoding='utf-8')
df.to_json(json_file, orient="records", force_ascii=False)
df

,date,country,countryCode,province,provinceCode,city,cityCode,confirmed,suspected,cured,dead
1,2020-01-13,中国,,,,,,41,0,0,1
2,2020-01-14,中国,,,,,,41,0,0,1
3,2020-01-15,中国,,,,,,41,0,5,2
4,2020-01-16,中国,,,,,,45,0,8,2
5,2020-01-17,中国,,,,,,62,0,12,2
...,...,...,...,...,...,...,...,...,...,...,...
220,2020-01-27,澳大利亚,,,,,,5,0,0,0
216,2020-01-27,美国,,,,,,5,0,0,0
218,2020-01-27,越南,,,,,,2,0,0,0
215,2020-01-27,韩国,,,,,,4,0,0,0


In [43]:
import re
import math
from functools import lru_cache
import pandas as pd

csv_file = "Wuhan-2019-nCoV.csv"

country_code = pd.read_csv("CountryCode.csv")
china_area_code = pd.read_csv("ChinaAreaCode.csv")
china_area_code["code"] = china_area_code["code"].astype(str)
# china_area_code = china_area_code[china_area_code.apply(lambda x: bool(re.match("\\d{4}00$", x.code)), axis=1)]
china_area_code["is_province"] = china_area_code["code"].map(lambda x: bool(re.match("\\d{2}0000$", x)))
# china_area_code["city_code"] = china_area_code["code"].map(lambda x: re.sub("\\d{2}$", "00", x))
china_area_code["province_code"] = china_area_code["code"].map(lambda x: re.sub("\\d{4}$", "0000", x))

@lru_cache(maxsize = 128)
def get_country_code(name):
    result =  country_code.loc[country_code["name"].isin([name])]["code"]
    if (len(result.values) > 0):
        return result.values[0]
    return ""


@lru_cache(maxsize = 32)
def get_china_province_code(name):
    if not name:
        return ""
    result = china_area_code.loc[china_area_code["is_province"] & china_area_code["name"].str.contains(name)]["code"]
    if (len(result.values) > 0):
        return result.values[0]
    return ""


# @lru_cache(maxsize = 1024)
def get_china_city_code(province_code, name):
    if not name or not province_code:
        return ""
    result = china_area_code.loc[china_area_code["province_code"].isin([province_code]) & ~china_area_code["is_province"] & china_area_code["name"].str.contains(name)]["code"]
    if (len(result.values) > 0):
        return result.values[0]

    for i in range(1, len(name)):
        fuzzy_name = name[:-i] + ".*" + ".*".join(name[-i:])
        result = china_area_code.loc[china_area_code["province_code"].isin([province_code]) & ~china_area_code["is_province"] & china_area_code["name"].str.match(fuzzy_name)]["code"]
        if (len(result.values) > 0):
            print(f"""{province_code} {fuzzy_name} -> {",".join(result.values)}""")
            return result.values[0]

    return ""


@lru_cache(maxsize = 1024)
def get_china_area_name(code, name):
    if not code:
        return name
    result = china_area_code.loc[china_area_code["code"].isin([code])]["name"]
    if (len(result.values) > 0):
        return result.values[0]
    return name

df = pd.read_csv(csv_file)
df["country"].fillna("", inplace=True)
df["countryCode"].fillna("", inplace=True)
df["province"].fillna("", inplace=True)
df["provinceCode"].fillna("", inplace=True)
df["city"].fillna("", inplace=True)
df["cityCode"].fillna("", inplace=True)
df["countryCode"] = df["country"].map(get_country_code)
df["provinceCode"] = df["province"].map(get_china_province_code)
df["province"] = df.apply(lambda x: get_china_area_name(x.provinceCode, x.province), axis = 1)
df["cityCode"] = df.apply(lambda x: get_china_city_code(x.provinceCode, x.city), axis = 1)
df["city"] = df.apply(lambda x: get_china_area_name(x.cityCode, x.city), axis = 1)
df.to_csv("temp.csv", index=False)
df

In [44]:
get_china_city_code("220000", "吉林")

'220200'

In [1]:
import math
s = "黔南州"
n = math.ceil(len(s) / 2)
for i in range(1, n):
    print(s[:-i])

黔南


In [32]:
import requests
import json
from datetime import datetime
import pandas as pd

toutiao_forum = requests.get("https://i.snssdk.com/forum/home/v1/info/?forum_id=1656784762444839").json()
toutiao_data = json.loads(toutiao_forum["forum"]["extra"]["ncov_string_list"])

datalist = []

dataDate = datetime.fromtimestamp(toutiao_data["updateTime"]).strftime('%Y-%m-%d')
for province in toutiao_data["provinces"]:
    datalist.append({
        "date": dataDate,
        "countryCode": "CN",
        "country": "中国",
        "provinceCode": province["id"].ljust(6, '0'),
        "province": province["name"],
        "cityCode": "",
        "city": "",
        "confirmed": province["confirmedNum"],
        "suspected": 0,
        "cured": province["curesNum"],
        "dead": province["deathsNum"]
    })
    for city in province["cities"]:
        datalist.append({
            "date": dataDate,
            "countryCode": "CN",
            "country": "中国",
            "provinceCode": province["id"].ljust(6, '0'),
            "province": province["name"],
            "cityCode": city["id"].ljust(6, '0'),
            "city": city["name"],
            "confirmed": city["confirmedNum"],
            "suspected": 0,
            "cured": city["curesNum"],
            "dead": city["deathsNum"]
        })
    for province_history in province["series"]:
        datalist.append({
            "date": province_history["date"],
            "countryCode": "CN",
            "country": "中国",
            "provinceCode": province["id"].ljust(6, '0'),
            "province": province["name"],
            "cityCode": "",
            "city": "",
            "confirmed": province_history["confirmedNum"],
            "suspected": 0,
            "cured": province_history["curesNum"],
            "dead": province_history["deathsNum"]
        })

for cn in toutiao_data["nationwide"]:
    datalist.append({
        "date": cn["date"],
        "countryCode": "CN",
        "country": "中国",
        "provinceCode": "",
        "province": "",
        "cityCode": "",
        "city": "",
        "confirmed": cn["confirmedNum"],
        "suspected": cn["suspectedNum"],
        "cured": cn["curesNum"],
        "dead": cn["deathsNum"]
    })

for country in toutiao_data["world"]:
    datalist.append({
        "date": dataDate,
        "country": country["country"],
        "confirmed": country["confirmedNum"],
        "suspected": country["suspectedNum"],
        "cured": country["curesNum"],
        "dead": country["deathsNum"]
    })


df = pd.DataFrame(datalist)
df


,date,countryCode,country,provinceCode,province,cityCode,city,confirmed,suspected,cured,dead
0,2020-02-06,CN,中国,420000,湖北,,,19665,0,677,549
1,2020-02-06,CN,中国,420000,湖北,420100,武汉,10117,0,413,414
2,2020-02-06,CN,中国,420000,湖北,420900,孝感,1886,0,9,25
3,2020-02-06,CN,中国,420000,湖北,421100,黄冈,1807,0,62,29
4,2020-02-06,CN,中国,420000,湖北,421300,随州,834,0,9,9
...,...,...,...,...,...,...,...,...,...,...,...
1015,2020-02-06,NaN,斯里兰卡,NaN,NaN,NaN,NaN,1,0,1,0
1016,2020-02-06,NaN,芬兰,NaN,NaN,NaN,NaN,1,0,0,0
1017,2020-02-06,NaN,瑞典,NaN,NaN,NaN,NaN,1,0,0,0
1018,2020-02-06,NaN,西班牙,NaN,NaN,NaN,NaN,1,0,0,0
